In [21]:
# CREATE EXTENSION plpythonu;


# CREATE OR REPLACE FUNCTION execute_python_script(python_script text)
# RETURNS text
# LANGUAGE plpythonu
# AS $$
# # Execute the Python script
# exec_result = plpy.execute(python_script, 1)

# return exec_result[0]["?column?"]
# $$;


In [22]:
# import psycopg2

# # Replace these with your database connection details
# db_host = 'localhost'
# db_name = 'TestDataBase'
# db_user = 'postgres'
# db_password = '2357'

# python_script = """
# print("Hello from Python script!")
# """

# try:
#     # Connect to the database
#     conn = psycopg2.connect(
#         host=db_host,
#         database=db_name,
#         user=db_user,
#         password=db_password
#     )

#     # Create a cursor
#     cursor = conn.cursor()

#     # Define the function call
#     function_call = "SELECT * from Employees"
# # execute_python_script('{python_script}')"

#     # Execute the function
#     cursor.execute(function_call)

#     # Fetch the result
#     result = cursor.fetchone()

#     # Print the result
#     print("Result from PostgreSQL:", result)

#     # Close the cursor and the connection
#     cursor.close()
#     conn.close()

# except (Exception, psycopg2.Error) as error:
#     print("Error while connecting to PostgreSQL:", error)


In [23]:
import openai

import json

import psycopg2

import os

import ast

import re

import time

import datetime

from contextlib import suppress

openai.api_key= '46d7ed565dda46f1893e7d0e42b3aefb'
openai.api_base = 'https://openaiautomationteam.openai.azure.com/'
openai.api_type= 'azure'
TEMPERATURE = 0.7


# STOP = os.getenv('STOP', "")
# MAX_TOKENS = float(os.getenv('MAX_TOKENS', 500))
# TOP_P = int(os.environ.get('TOP_P', 1))
# FREQUENCY_PENALTY = int(os.getenv('FREQUENCY_PENALTY', 0))
# PRESENCE_PENALTY = int(os.getenv('PRESENCE_PENALTY', 0))
# N_RESP = int(os.getenv('N_RESP', 1))
# TIMEOUT = int(os.getenv('TIMEOUT', 60))

DB_HOST = os.environ.get('DB_HOST', None)
DB_PORT = os.environ.get('DB_PORT', None)
DB_USER = os.environ.get('DB_USER', None)
DB_PASSWORD = os.environ.get('DB_PASSWORD', None)
DB_NAME = os.environ.get('DB_NAME', None)


In [24]:

PGRESCREDS={'Host': DB_HOST, 'Port': DB_PORT, 'User': DB_USER, 'Password': DB_PASSWORD, 'Name': DB_NAME}

Creds = list(PGRESCREDS.values())

sql={"sql":"SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'public';"}


sql_key = list(sql.keys())

In [25]:
def automate_function_call(Creds, sql_key, response_message):
    '''Function calling for Running the generated python code on PostGRES Server'''


    conn = psycopg2.connect(host=Creds[0], database=Creds[4], user=Creds[2], password=Creds[3], port=Creds[1])
    
    def extract_outermost_dict(s):
        for i in range(len(s)):
            for j in range(i, len(s)):
                substring = s[i:j+1]
                try:
                    node = ast.literal_eval(substring)
                    if isinstance(node, dict):
                        return node
                except (ValueError, SyntaxError):
                    pass
        return None
    
    def extract_json(response_message):
        """Function to extract JSON from the output response. The JSON is within triple bacticks

        Args:
            output_response (str): output response

        Returns:
            dict: json
        """
        json_output = re.findall(r"```([\s\S]*?)```", response_message)

        if len(json_output) == 0:
            return ''
        elif len(json_output) >= 1:
            json_output = json_output[0]
        
        json_output = extract_outermost_dict(json_output)
        return json_output
    
    def validate_json(self, input_json):
        return True, "The JSON is valid"

    json_output = extract_json(response_message)

    def ExecuteSQL(sql_key):
        """Function to execute SQL query

        Args:
            sql (str): sql query to execute

        Returns:
            str: output of sql query
        """
        try:
            sql = sql[sql_key[0]] #key
            cur = conn.cursor()
            # Execute a SQL query
            cur.execute(sql)

            # Fetch the results
            results = cur.fetchall()

            # Close the cursor and connection
            cur.close()
            if results == '':
                return "PostgresSQL Query executed Successfully"
            else:
                ShareOutput(results)
                return results
        except Exception as e:
            return f"There is some error in SQL query: {str(e)}"
    
    def get_database_info(sql_key):
        """Function to get database and table information using sql queries

        Returns:
            dict: database info
        """
        try:
            result = ExecuteSQL(sql_key)
            update_history({"function":"ExecuteSQL","parameters":{"sql":"SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'public';"}}, result)
        except Exception as e:
            raise e

    def ShareOutput(output: str) -> str:
        """Function to share output

        Args:
            output (str): output to share

        Returns:
            str: output of sharing output
        """
        try:
            return output
        except Exception as e:
            return e

    def update_history(input_json: str, output: str) -> str:
        """
        Function to update history

        Args:
            input_prompt (str): input prompt
            output (str): output

        Returns:
            str: output of updating history
        """
        try:
            history = prepare_history(input_json, output)
            if history == "Empty":
                history = [history]
            else:
                history.append(history)
            
            for ind, his in enumerate(history, start=1):
                history += f"{ind}. {his}\n"

            user_prompt = base_user_prompt.replace('<history>', f"[{history}]")
        except Exception as e:
            raise e
        
    def prepare_history(input_json: str, output: str) -> str:
        """Function to prepare history

        Args:
            input_prompt (str): input prompt
            output (str): output

        Returns:
            str: output of preparing history
        """
        try:
            return f"Assistant Response-\n{input_json}\nExecution Output-\n{output}\n"
        except Exception as e:
            raise e


    get_database_info(sql_key)

    time.sleep(2)

    if json_output == '':
        print(response_message)
        # break

    valid_json, output = validate_json(json_output)


    if not valid_json:
        update_history(response_message, output)
        # continue

    function_to_perform = json_output['function']

    if 'parameters' in json_output:
        function_params = json_output['parameters']

    # if function_to_perform == "Exit":
        # break

    supported_functions = {
            "ExecuteSQL": ExecuteSQL,
            "ShareOutput": ShareOutput
        }
    
    output = supported_functions[function_to_perform](function_params)
    print(output)
    
    update_history(response_message, output)
    
    arguments = {

        "Creds": Creds,

        "sql_key": sql_key,

        "response_message": response_message,

    }
    
    json.dumps(arguments)
    
    return output


    

In [26]:
input_prompt="Generate a directly python script to plot graphs for a given dataset: 'Data.xlsx' and an explanation of observations in each graph. Give explanation as a paragraph after every graph."

In [27]:
import pandas as pd
data=pd.read_excel("Data.xlsx")
columns=data.columns

In [28]:
des=data.describe()

In [29]:
input_prompt="Generate a directly python script to plot graphs for a given dataset: 'Data.xlsx' and an explanation of observations in each graph. Give explanation as a paragraph after every graph."

In [30]:
user_prompt = input_prompt+ "; " + f"Columns: {columns}" +"; "+f"Dataset Insights: {des}"
user_prompt

"Generate a directly python script to plot graphs for a given dataset: 'Data.xlsx' and an explanation of observations in each graph. Give explanation as a paragraph after every graph.; Columns: Index(['Sales_Rep', 'Business', 'Age', 'Female', 'Years', 'College',\n       'Personality', 'Certficates', 'Feedback', 'Salary',\n       'customer-satisfaction-score'],\n      dtype='object'); Dataset Insights:           Sales_Rep           Age        Female         Years   Certficates  \\\ncount  21990.000000  21990.000000  21990.000000  21990.000000  21990.000000   \nmean   10995.500000     41.495953      0.383038      2.646385      2.612187   \nstd     6348.110546     11.413748      0.486138      2.434109      1.648258   \nmin        1.000000     21.000000      0.000000      1.000000      0.000000   \n25%     5498.250000     32.000000      0.000000      1.000000      1.000000   \n50%    10995.500000     41.000000      0.000000      2.000000      2.000000   \n75%    16492.750000     51.000000 

In [31]:
prompt = user_prompt + '''Make sure that every plot gives some major insights of the dataset. Don't use scatter plot for comparison of two features.'''

In [32]:

system_prompt = '''You are a data analyst. Call a function 'automate_function_call' which will execute the python code generated to Postgres server and give us the output after execution.'''



In [33]:
functions = [
        {
            "name": "automate_function_call",
            "description": "Automate the function calling in such a way that history is updated",
            "parameters": {
                "type": "object",
                "properties": {
                    "Creds": {
                        "type": "list",
                        "description": "Credentials for PostGRES which are used in the function using psycopg2 library",
                    },
                    "sql_key": {
                        "type": "string",
                        "description": "The key for the SQL Query to be executed  "
                    },
                    "response_message": {
                        "type": "string",
                        "description": "First response of GPT API for system_prompt and user_prompt"
                    },
                },
                "required": ["Creds", "sql_key", "response_message"]
            }
        }, 

    ]

messages =[{"role":"system", "content":system_prompt}, {"role":"user", "content":prompt}]


result_GPT = openai.ChatCompletion.create(
    
    engine='DIR_GPT4',
    temperature=0.7,
    messages=messages,
    functions=functions,
)

response_message = result_GPT.choices[0].message.content

print(response_message)
    

InvalidRequestError: Unrecognized request argument supplied: functions

In [ ]:
if response_message.get("function_call"):

    available_functions = {

        "automate_function_call": automate_function_call,

    }

    function_name = response_message["function_call"]["name"]

    fuction_to_call = available_functions[function_name]

    function_args = json.loads(response_message["function_call"]["arguments"])

    function_response = fuction_to_call(


        Creds=function_args.get("Creds"),

        sql_key=function_args.get("sql_key"),

        response_message=function_args.get("response_message"),

    )

    messages.append(response_message)

    messages.append(

        {

            "role": "function",

            "name": function_name,

            "content": function_response,

        }

    )

    second_response = openai.ChatCompletion.create(

        engine="DIR_GPT4",

        messages=messages,

    )

print(second_response)

NameError: name 'response_message' is not defined